# Generative Performance
## Fitting models to player 2 (Trustee)

In [27]:
import sys
sys.path.append('/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/models/')
sys.path.append('/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/')
import pandas as pd
import numpy as np
from scipy.optimize import least_squares
import p2_cost_funcs
import comp_models
from utils import  folk_params, compute_aic, compute_bic


dat = pd.read_csv('../data/bx_data.csv', index_col=0)

results = pd.DataFrame(columns=['p2', 'p3', 'p2_strat', 'model', 'param1', 'param2', 'sse', 'aic', 'bic', 'fit_method'])



niters = 10


## Fit models to p2 to simulate different agents performing task

In [29]:
for turk in dat.turker_id.unique():
    game = dat.loc[dat.turker_id == turk].reset_index(drop=True)
    #Fit Rescorla Wagner
    model = 'rescorla-wagner'
    print('Fitting '+model+' model to p2 for turker: '+turk+'.')

    fitIters = np.zeros([niters, 5])
    for i in range(niters):
        param0 = np.random.uniform()
        fitIters[i,0:2] = [param0, np.nan]
        result_lsq = least_squares(fun=p2_cost_funcs.rw_costfun, x0=param0, args=(game, ), bounds=(0,1))

        param1 = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, np.nan, cost]
    cost_selected = np.min(fitIters[:, 4])
    alpha = fitIters[fitIters[:,4]== cost_selected, 2][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, alpha,np.nan, sse, compute_aic(len(game), sse, 1), compute_bic(len(game), sse, 1),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Inverse Motivation Model
    model = 'inverse motive'
    print('Fitting '+model+' model to p2 for turker: '+turk+'.')
    fitIters = np.zeros([niters, 5])
    for i in range(niters):
        param0 = [np.random.uniform(), np.random.uniform(low=.001)]
        fitIters[i,0:2] = [param0[0], param0[1]]
        result_lsq = least_squares(fun=p2_cost_funcs.imm_costfun, x0=param0, args=(game, folk_params ), bounds=([0,.001], [1,1]))

        [param1, param2] = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, param2, cost]
    cost_selected = np.min(fitIters[:, 4])
    alpha = fitIters[fitIters[:,4]== cost_selected, 2][0]
    tau = fitIters[fitIters[:,4]== cost_selected, 3][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, alpha, tau, sse, compute_aic(len(game), sse, 2), compute_bic(len(game), sse, 2),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    # "Fit" Greed Model
    model = 'greed'
    print('Fitting '+model+' model to p2 for turker: '+turk+'.')
    sse = sum((p2_cost_funcs.gr_costfun(game))**2)
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, np.nan, np.nan, sse, compute_aic(len(game), sse, 0), compute_bic(len(game), sse, 0),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Guilt-aversion model
    model = 'guilt-aversion'
    print('Fitting '+model+' model to p2 for turker: '+turk+'.')
    for i in range(niters):
        param0 = np.random.uniform()
        fitIters[i,0:2] = [param0, np.nan]
        result_lsq = least_squares(fun=p2_cost_funcs.ga_costfun, x0=param0, args=(game, ), bounds=(0,1))

        param1 = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, np.nan, cost]
    cost_selected = np.min(fitIters[:, 4])
    theta = fitIters[fitIters[:,4]== cost_selected, 2][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, theta, np.nan, sse, compute_aic(len(game), sse, 1), compute_bic(len(game), sse, 1),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Inequity-aversion model
    model = 'inequity-aversion'
    print('Fitting '+model+' model to p2 for turker: '+turk+'.')
    for i in range(niters):
        param0 = np.random.uniform()
        fitIters[i,0:2] = [param0, np.nan]
        result_lsq = least_squares(fun=p2_cost_funcs.ia_costfun, x0=param0, args=(game, ), bounds=(0,1))

        param1 = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, np.nan, cost]
    cost_selected = np.min(fitIters[:, 4])
    theta = fitIters[fitIters[:,4]== cost_selected, 2][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, theta, np.nan, sse, compute_aic(len(game), sse, 1), compute_bic(len(game), sse, 1),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Moral Phenotype model
    model = 'moral phenotype'
    print('Fitting '+model+' model to p2 for turker: '+turk+'.')
    fitIters = np.zeros([niters, 5])
    for i in range(niters):
        param0 = [np.random.uniform()/2, np.random.uniform()/5-0.1]
        fitIters[i,0:2] = [param0[0], param0[1]]
        result_lsq = least_squares(fun=p2_cost_funcs.mp_costfun, x0=param0, args=(game, ), bounds=([0,-.1], [.5,.1]))

        [param1, param2] = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, param2, cost]
    cost_selected = np.min(fitIters[:, 4])
    theyta = fitIters[fitIters[:,4]== cost_selected, 2][0]
    phee = fitIters[fitIters[:,4]== cost_selected, 3][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, theyta, phee, sse, compute_aic(len(game), sse, 2), compute_bic(len(game), sse, 2),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))
